# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
import openai 
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [5]:
%pip install --upgrade openai


Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai
print(openai.__version__)


0.28.0


In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an expert in F1."},
        {"role": "user", "content": "Who won the F1 2010?"}
    ],
    temperature=0.7
)

print(response.choices[0].message.content)


Sebastian Vettel won the 2010 Formula 1 World Championship driving for Red Bull Racing.


In [5]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    #api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [6]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010. He was driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [7]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [8]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The F1 2006 championship was won by Fernando Alonso driving for Renault.


In [9]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [10]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [11]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [13]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

Zero Shot

In [14]:
context_resume_zero = [
    {'role': 'system', 'content': 'You are a career advisor that reviews resumes and gives suggestions for improvement in a helpful, professional tone.'}
]

resume_text = """John Doe
Software Engineer with 3 years of experience in full-stack development. Skilled in Python, React, and AWS. Looking for a challenging role in a fast-paced tech environment."""

print(return_OAIResponse(f"Can you review this resume summary?\n{resume_text}", context_resume_zero))


Your resume summary is clear and concise, highlighting your experience and technical skills effectively. To enhance it further, consider adding specific accomplishments or projects that demonstrate your expertise in Python, React, and AWS. Providing quantifiable results or successful implementations can make your summary more impactful and showcase your abilities to potential employers. Additionally, be sure to tailor your summary to the specific job you are applying for by aligning your skills and experiences with the requirements of the role. Good luck with your job search!


One Shot

In [15]:
context_resume_one = [
    {'role': 'system', 'content': 'You are a career advisor that reviews resumes and gives suggestions for improvement in a helpful, professional tone.'},
    {'role': 'user', 'content': 'Here is my resume summary: Jane Smith\nExperienced marketing specialist with 5 years in digital advertising and social media strategy. Seeking to lead impactful campaigns in a dynamic team.'},
    {'role': 'assistant', 'content': 'Your summary is clear and focused. To make it stronger, consider quantifying your achievements, like “led a social media campaign that increased engagement by 40%.” That helps recruiters see your impact.'}
]

resume_input = """Michael Lee
Mechanical Engineer with 6 years of experience in product design and thermal systems. Interested in sustainable energy solutions."""

print(return_OAIResponse(f"Here is my resume summary: {resume_input}", context_resume_one))


Your summary effectively highlights your experience and interest in sustainable energy solutions. To enhance it, consider adding specific achievements like “designed a new product that reduced production costs by 20%.” This will showcase your impact and abilities.


Two Shot

In [16]:
context_resume_two = [
    {'role': 'system', 'content': 'You are a career advisor that reviews resumes and gives suggestions for improvement in a helpful, professional tone.'},
    {'role': 'user', 'content': 'Here is my resume summary: Sarah Khan\nProject manager with 7 years in construction and infrastructure. Delivered projects on-time and under budget.'},
    {'role': 'assistant', 'content': 'This is a solid start! You could improve it by specifying the size or scope of projects you’ve handled — e.g., “managed $5M+ infrastructure projects” — and add key certifications if any (like PMP).'},
    {'role': 'user', 'content': 'Here is my resume summary: Rahul Mehta\nData analyst with experience in SQL, Excel, and Tableau. Passionate about turning data into business insights.'},
    {'role': 'assistant', 'content': 'You can enhance this by showing results. For example, “built dashboards that improved reporting speed by 30%.” Employers value impact.'}
]

resume_input = """Emily Turner
UX designer passionate about creating seamless and user-friendly web and mobile experiences. Skilled in Figma, Adobe XD, and user research."""

print(return_OAIResponse(f"Here is my resume summary: {resume_input}", context_resume_two))


This summary effectively highlights your expertise and passion. Consider adding quantifiable achievements like “Increased user engagement by 20% through redesign of mobile app interface.” This would provide concrete evidence of your skills in action.


Report on Resume Review Assistant Prompt Experiments

Findings

I tested zero-shot, one-shot, and two-shot prompts for a resume review assistant to see how the AI’s responses varied.

    Zero-shot: The AI gave basic, general feedback but sometimes lacked detail or practical suggestions.

    One-shot: Adding one example improved specificity and made the advice more useful, though some phrasing was repetitive.

    Two-shot: Two examples led to richer, more tailored responses but sometimes became too verbose or vague.

Challenges

    The AI occasionally made up details not in the resume (hallucinations).

    Some answers repeated wording from examples.

    Zero-shot prompts missed key points like formatting or soft skills.

Lessons Learned

Including examples helps the AI give better, more relevant feedback. However, responses should be checked carefully for errors or made-up content. Balancing prompt length is important to get clear, useful advice without too much repetition.